<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/038_JW_Modular_AgentDesign_II_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 Building a Simple Agent Framework

Now, we are going to put the components together into a **reusable agent class**. This class encapsulates the GAME components and provides a clean interface for running the agent loop. We can create different agents simply by changing the **Goals**, **Actions**, **Memory**, and **Environment** (GAME) without modifying the core loop.

In [ ]:
class Agent:
    def __init__(self,
                 goals: List[Goal],
                 agent_language: AgentLanguage,
                 action_registry: ActionRegistry,
                 generate_response: Callable[[Prompt], str],
                 environment: Environment):
        """
        Initialize an agent with its core GAME components
        """
        self.goals = goals
        self.generate_response = generate_response
        self.agent_language = agent_language
        self.actions = action_registry
        self.environment = environment

    def construct_prompt(self, goals: List[Goal], memory: Memory, actions: ActionRegistry) -> Prompt:
        """Build prompt with memory context"""
        return self.agent_language.construct_prompt(
            actions=actions.get_actions(),
            environment=self.environment,
            goals=goals,
            memory=memory
        )

    def get_action(self, response):
        invocation = self.agent_language.parse_response(response)
        action = self.actions.get_action(invocation["tool"])
        return action, invocation

    def should_terminate(self, response: str) -> bool:
        action_def, _ = self.get_action(response)
        return action_def.terminal

    def set_current_task(self, memory: Memory, task: str):
        memory.add_memory({"type": "user", "content": task})

    def update_memory(self, memory: Memory, response: str, result: dict):
        """
        Update memory with the agent's decision and the environment's response.
        """
        new_memories = [
            {"type": "assistant", "content": response},
            {"type": "user", "content": json.dumps(result)}
        ]
        for m in new_memories:
            memory.add_memory(m)

    def prompt_llm_for_action(self, full_prompt: Prompt) -> str:
        response = self.generate_response(full_prompt)
        return response

    def run(self, user_input: str, memory=None, max_iterations: int = 50) -> Memory:
        """
        Execute the GAME loop for this agent with a maximum iteration limit.
        """
        memory = memory or Memory()
        self.set_current_task(memory, user_input)

        for _ in range(max_iterations):
            # Construct a prompt that includes the Goals, Actions, and the current Memory
            prompt = self.construct_prompt(self.goals, memory, self.actions)

            print("Agent thinking...")
            # Generate a response from the agent
            response = self.prompt_llm_for_action(prompt)
            print(f"Agent Decision: {response}")

            # Determine which action the agent wants to execute
            action, invocation = self.get_action(response)

            # Execute the action in the environment
            result = self.environment.execute_action(action, invocation["args"])
            print(f"Action Result: {result}")

            # Update the agent's memory with information about what happened
            self.update_memory(memory, response, result)

            # Check if the agent has decided to terminate
            if self.should_terminate(response):
                break

        return memory




# 🧠 Simple Agent Framework — What to Learn & Watch For

## 0) Mental model (GAME)

This Agent is organized around **G**oals, **A**ctions, **M**emory, and **E**nvironment:

* **Goals**: what the agent is trying to achieve (instructions/constraints).
* **Actions**: the tools the agent may call (wrapped in your `Action` objects + `ActionRegistry`).
* **Memory**: the evolving transcript/state (what was asked, what the model decided, tool results).
* **Environment**: the executor/sandbox that *runs* actions and returns results.

The Agent orchestrates these in a loop.

---

## 1) Constructor (dependencies, not logic)

```python
class Agent:
    def __init__(self, goals, agent_language, action_registry, generate_response, environment):
        self.goals = goals
        self.generate_response = generate_response
        self.agent_language = agent_language
        self.actions = action_registry
        self.environment = environment
```

### What to focus on

* **Dependency injection**: you pass in everything (LLM caller, registry, environment). This makes it **testable and modular**.
* `agent_language` is a key abstraction:

  * `construct_prompt(...)`: how to turn Goals/Actions/Memory into model input.
  * `parse_response(...)`: how to read the model’s output into `{"tool": name, "args": {...}}` (or a terminal answer).
  * This lets you swap prompt formats (plain text vs. OpenAI tool-calling) **without changing the Agent**.

---

## 2) Prompt construction

```python
def construct_prompt(self, goals, memory, actions) -> Prompt:
    return self.agent_language.construct_prompt(
        actions=actions.get_actions(),
        environment=self.environment,
        goals=goals,
        memory=memory
    )
```

### What to focus on

* Keep prompts **deterministic and short**. Only include what the model needs: goals, tool list (names + short descriptions + schemas), and the **relevant** memory.
* For long runs, add a **memory summarizer** to avoid context blowup.

---

## 3) Parsing the model’s decision

```python
def get_action(self, response):
    invocation = self.agent_language.parse_response(response)
    action = self.actions.get_action(invocation["tool"])
    return action, invocation
```

### What to focus on

* `parse_response` should return a **normalized invocation** like:

  ```python
  {"tool": "read_file", "args": {"file_name": "notes.txt"}}
  ```
* **Validation goes here** (or right after): check tool name whitelist, schema (types/required keys), and **semantic rules** (e.g., path safety).
* If you adopt **OpenAI tool calling**, `parse_response` should read `message.tool_calls[*]` instead of scraping text.

---

## 4) Termination

```python
def should_terminate(self, response: str) -> bool:
    action_def, _ = self.get_action(response)
    return action_def.terminal
```

### What to focus on

* You’ll typically have a special action like `final_answer` with `terminal=True`.
* Alternatively, let `agent_language.parse_response` return a flag like `{"tool":"final_answer", "args":{...}, "terminal": True}`.

---

## 5) Memory updates

```python
def set_current_task(self, memory, task):
    memory.add_memory({"type": "user", "content": task})

def update_memory(self, memory, response, result):
    new_memories = [
        {"type": "assistant", "content": response},
        {"type": "user", "content": json.dumps(result)}
    ]
    for m in new_memories:
        memory.add_memory(m)
```

### What to focus on

* Keep roles consistent (e.g., `"user"`, `"assistant"`, `"tool"`). Consider storing tool results under a **tool role** or a structured entry like `{"type":"tool","name":action.name,"result":...}`.
* Watch **memory growth**: summarize or window past steps.
* Store **structured** results when possible (JSON) to enable downstream reasoning.

---

## 6) The run loop (the engine)

```python
def run(self, user_input, memory=None, max_iterations=50) -> Memory:
    memory = memory or Memory()
    self.set_current_task(memory, user_input)

    for _ in range(max_iterations):
        prompt = self.construct_prompt(self.goals, memory, self.actions)
        print("Agent thinking...")
        response = self.prompt_llm_for_action(prompt)
        print(f"Agent Decision: {response}")

        action, invocation = self.get_action(response)
        result = self.environment.execute_action(action, invocation["args"])
        print(f"Action Result: {result}")

        self.update_memory(memory, response, result)

        if self.should_terminate(response):
            break

    return memory
```

### What to focus on

* **Separation of concerns**:

  * Agent builds prompt → LLM decides → Registry resolves tool → Environment executes → Memory records → Loop decides to continue.
* **Where guardrails live**:

  * Before `execute_action`: **validate invocation args** (schema + business rules).
  * In `environment.execute_action`: sandboxing, path whitelists, network policy, timeouts, idempotency.
  * Add **retry-on-parse-fail** (≤1–2) with stricter instructions/temperature 0.0.
* **Observability**:

  * Log prompts, tool calls, results, tokens/costs, iterations, errors. Add a circuit breaker if failures spike.
* **Max iterations**: prevents infinite loops. Many agents also track a **budget** (token/cost/time).

---

## 7) Key features to learn from this architecture

1. **Pluggable layers**

   * Swap the LLM (`generate_response`), prompt/parse logic (`agent_language`), tools (`ActionRegistry`), or execution (`Environment`) independently.

2. **Action contract**

   * Each tool has a **name, description, JSON Schema**, and a Python function. This mirrors provider APIs and supports validation.

3. **Structured decisions**

   * Model output becomes a **machine-readable invocation** (`tool` + `args`), not free-form text. That’s what makes it an *agent*, not just a chatbot.

4. **Memory as context**

   * The loop builds on past steps. Learn when to **summarize**, **filter**, or **pin** critical facts to keep the context small and relevant.

5. **Termination & control**

   * Clear end conditions (terminal action or goal satisfied). Avoid unbounded loops.

6. **Safety & validation**

   * Treat model output as untrusted: **schema-validate** + **semantic-validate** *before* executing anything with side effects.

---

## 8) Pragmatic upgrades you can add

* **OpenAI tool calling**: move argument parsing from text to `message.tool_calls[*]`.
* **Pydantic**: validate invocations (`tool_name` enums; typed arg models; custom validators).
* **Retry-on-parse**: if `parse_response` fails, append a corrective message and try once more at `temperature=0`.
* **Result shaping**: make tools return **structured JSON** for better downstream reasoning.
* **Memory policy**: sliding window + periodic summary nodes (map-reduce style).
* **Metrics**: count steps, tool-call success rate, invalid-invocation rate.

---

## 9) What the grader/interviewer will care about

* Clear **boundaries** between planning (LLM) and acting (Python/Environment).
* Robust **validation** and **guardrails**.
* Ability to **swap** prompting strategy (e.g., JSON-mode/tool-calling) without rewriting the agent.
* Sensible **termination** and **memory management**.

---

### TL;DR

Focus on the **interfaces** between: Prompt ↔️ LLM ↔️ Invocation ↔️ Registry ↔️ Environment ↔️ Memory.
If each boundary is clean and validated, your agent will be reliable, extensible, and safe.




## 🧠 What Is `AgentLanguage`?

The `AgentLanguage` class is a **core abstraction** responsible for:

* 📝 **Formatting the prompt** that gets sent to the LLM
* 📦 **Parsing the model’s response** into structured tool calls

It acts as the “language layer” that determines how your agent **talks to** and **interprets responses from** the LLM.

---

## ✅ How `AgentLanguage` Is Used in the Agent Loop

### 🔹 Step 1: Constructing the Prompt

When the agent loop begins, it first constructs the LLM prompt like this:

```python
def construct_prompt(self, goals: List[Goal], memory: Memory, actions: ActionRegistry) -> Prompt:
    return self.agent_language.construct_prompt(
        actions=actions.get_actions(),
        environment=self.environment,
        goals=goals,
        memory=memory
    )
```

This method builds a **structured input** for the model using four key components:

| Component      | Description                                                             |
| -------------- | ----------------------------------------------------------------------- |
| 🧭 Goals       | What the agent is trying to accomplish                                  |
| 🛠️ Actions    | The tools or functions available to the agent                           |
| 🧠 Memory      | Prior messages, file contents, and context relevant to the current task |
| 🌍 Environment | Constraints, settings, or metadata about where the agent is running     |

---

## 🔄 Parsing Responses: AgentLanguage Again

Later in the loop, after receiving the LLM response, `AgentLanguage` also handles **interpreting that response**:

```python
action, invocation = self.agent_language.parse_action_response(response)
```

If you're using **OpenAI function calling**, this is typically as simple as extracting the structured `tool_calls` block.

But by **decoupling** the formatting/parsing logic into `AgentLanguage`, you can:

* 🔁 Swap out OpenAI for another LLM format
* 📜 Use natural language responses instead of tool calls (for simulation)
* 🧪 Test your agent without hard-coding the LLM's output logic

---

## 🧩 Why This Matters

This modular `AgentLanguage` component lets your agent:

* Speak **different languages** (OpenAI function calling, plain text, etc.)
* Be tested or simulated in different contexts
* Separate **communication logic** from **business logic**

So even though we use OpenAI tools most of the time, this abstraction gives you a path to scale, extend, and simulate more flexibly.

---


## 🧠 **Memory Context**

**Definition**:
Memory is a record of everything the agent has "seen" or "done" so far in the current session.

### 🔍 Why it matters:

* It helps the agent **maintain continuity** over multiple steps.
* Without it, the agent would be stateless — it would forget past actions, results, or user instructions.

### 🧩 What's typically stored:

* The **user’s original request**
* Past **LLM responses**
* Previous **tool invocations**
* Results of those tools (e.g., content from a file it read)

### ✅ Example:

```python
[
  {"type": "user", "content": "Refactor the data processing script."},
  {"type": "assistant", "content": "I'll begin by listing files in the 'scripts/' directory."},
  {"type": "user", "content": "['clean.py', 'process.py']"},
  {"type": "assistant", "content": "Reading process.py now."},
  {"type": "user", "content": "# contents of process.py..."}
]
```

This history allows the agent to ask follow-up questions, avoid repeating steps, and remember relevant files or outputs.

---

## 🌍 **Environment Info**

**Definition**:
A description of the **context** in which the agent is operating. This is static info about the world the agent is working in.

### 📦 Typical contents:

* "You are working in a local Python project folder."
* "You have access to functions like `list_files()` and `read_file()`."
* "You do *not* have internet access."

### 🔐 Why it matters:

It **guides the agent’s reasoning boundaries**:

* Prevents hallucinating actions it cannot perform (like calling an API when it's not available).
* Sets correct assumptions about what data/tools are available.

### ✅ Example:

```text
You are operating in a local development environment.
You can read and write files, but cannot access the internet.
Use the tools provided below to complete the task.
```

---

## 💡 Summary

| Component        | Purpose                                           | Keeps Agent Aware Of     |
| ---------------- | ------------------------------------------------- | ------------------------ |
| Memory Context   | Conversation and execution history                | What’s already been done |
| Environment Info | Operational boundaries and available capabilities | What’s *possible* to do  |



### 🤖 Step 2: Generating a Response

After the prompt is constructed, the agent sends it to the language model:

```python
def prompt_llm_for_action(self, full_prompt: Prompt) -> str:
    response = self.generate_response(full_prompt)
    return response
```

### 🔧 What's Happening?

* `generate_response()` is an **injected function** defined during initialization.
* It handles the actual call to the LLM.
* This abstraction allows the agent framework to stay **model-agnostic**.

---

### 🤖 Why This Matters

This design provides flexibility:

* You can use **LiteLLM**, **OpenAI**, **Anthropic**, or any LLM.
* You don’t have to change the core agent loop to switch models.
* Great for mocking, testing, or deploying in environments with different LLMs.


### 🧠 Step 3: Parsing the Response

```python
action, invocation = self.get_action(response)
```

* `AgentLanguage` parses the model’s reply.
* `ActionRegistry` retrieves the action definition.
* `invocation` contains the tool name and argument values.

---

## ✅ What Is `AgentLanguage`?

`AgentLanguage` is a **custom abstraction** used in this lecture's framework to:

> Define how the LLM communicates actions to the agent.

It **controls how the LLM should express tool calls** in text, and how we **parse** those calls from its output.

### Think of `AgentLanguage` as:

* A bridge between **natural language** (from the LLM) and **structured commands** (the agent executes).
* A reusable module that **parses the LLM's output** into something your code can understand and act on.

### It likely provides:

* `format_action(action_name, args)` → Returns a prompt string to tell the LLM how to call the tool.
* `parse_action_response(response_text)` → Extracts `action_name` and `args` from the LLM’s text.

---

## 🧠 So What Is `get_action()` Doing?

In the agent loop:

```python
action, invocation = self.get_action(response)
```

Here’s what’s happening:

1. **`response`** = the raw LLM output (text).
2. `self.get_action()` internally calls something like:

   ```python
   return self.language.parse_action_response(response)
   ```
3. That parsing breaks the response into:

   * `action`: the name of the tool (e.g., `"read_python_file"`)
   * `invocation`: the actual argument dictionary (e.g., `{"file_name": "main.py"}`)

So **you don’t see `parse_response()` defined**, because it's likely wrapped inside this `AgentLanguage` object’s method.

---

## 🔁 Where Does This Fit in the Loop?

The structure is roughly:

```python
prompt = self.construct_prompt(...)
response = llm_call(prompt)
action, invocation = agent_language.parse_action_response(response)
tool_fn = registry[action]
tool_fn(**invocation)
```

---

## 📌 Summary

| Term            | Role                                                    |
| --------------- | ------------------------------------------------------- |
| `AgentLanguage` | Class that defines LLM input/output formatting          |
| `get_action()`  | Uses `AgentLanguage` to extract tool call from response |
| `invocation`    | Parsed tool arguments ready to call in Python           |





### 🔧 Step 4: Executing the Action

Once the tool and arguments are known, the agent performs the action:

```python
result = self.environment.execute_action(action, invocation["args"])
```

### 🛠️ Execution Happens in the Environment

* The `Environment` class is responsible for **carrying out the action**.
* This might involve:

  * Calling APIs
  * Accessing files
  * Running computations
  * Querying databases

### 🧩 Separation of Concerns

* The `ActionRegistry` knows **what** can be done.
* The `Environment` knows **how** to do it in the current context.







### 🧠 Step 5: Updating Memory

After the agent executes an action, it needs to **record what happened**—both the decision it made and the result of that action.

```python
def update_memory(self, memory: Memory, response: str, result: dict):
    """
    Update memory with the agent's decision and the environment's response.
    """
    new_memories = [
        {"type": "assistant", "content": response},             # What the LLM said
        {"type": "user", "content": json.dumps(result)}         # What the environment returned
    ]
    for m in new_memories:
        memory.add_memory(m)
```

### 💡 Why Update Memory?

* Keeps a **chronological record** of what the agent did and why.
* Memory becomes **part of the next prompt**, allowing the agent to reason across multiple steps.
* Helps the agent **avoid repetition**, reuse past insights, and improve coherence.

### 🧠 What Gets Stored?

1. The **LLM’s response** (usually a tool call or text-based reasoning).
2. The **environment’s result** (i.e., output from the executed action).

Together, this builds a conversational history that gets looped back into the LLM’s input, making the agent smarter over time.

---

### ✅ Here's what happens regarding token usage and cost:

#### 💾 1. **Memory is added to the prompt**

Each time the agent loops:

* It **constructs a new prompt**.
* This prompt includes:

  * The agent’s goals.
  * Available tool definitions (tool schemas).
  * The full **memory history** so far (unless truncated).
  * Possibly some environmental context.

#### 🔄 2. **All of that is sent to the LLM**

* OpenAI (and most LLM APIs) **charge based on token input and output**.
* So every piece of memory stored — whether it’s the user's request, the agent's tool call, or the tool's result — will be **converted into tokens** and **counted toward the prompt token budget**.

#### 💸 3. **You pay for it**

* You're billed for:

  * All **tokens in the request** (the full prompt).
  * All **tokens in the response** (e.g., a tool call or text reply).
* In long-running loops, the **memory can grow large**, increasing prompt size and cost quickly.

---

### 🧠 How to Manage This

* 🔄 **Summarize or compress memory**: Store only the essential decisions and results.
* 📉 **Truncate old context**: Only keep the last N exchanges.
* 🧱 **Structured memory**: Instead of dumping raw results, store simplified records (e.g., `"Searched files A, B, found match in B"`).
* 🧠 **Hybrid memory**: Use a vector store or database to retrieve memory selectively, instead of including all memory in every call.




### ⛔ Step 6: Termination Check

```python
if self.should_terminate(response): break
```

* Uses `action_def.terminal` to see if the agent should stop.


---

### 🧠 Why We Need a Termination Check

Agents usually run in loops:

1. Construct a prompt.
2. Call the LLM.
3. Decide what to do.
4. Execute a tool.
5. Update memory.
6. Repeat...

But this can’t go on forever. At some point, the agent must decide:

> “I’ve done everything I needed to do. Time to stop.”

That’s where the **`should_terminate()`** check comes in.

---

### 🔍 What the Code Does

```python
def should_terminate(self, response: str) -> bool:
    action_def, _ = self.get_action(response)
    return action_def.terminal
```

Here’s what this means, step by step:

1. **`get_action(response)`**
   → This parses the model’s response and returns:

   * `action_def`: the tool definition (from the registry)
   * `invocation`: the tool name + arguments

2. **`action_def.terminal`**
   → This is a Boolean property on the tool definition:

   * If `True`, this tool is a **terminal action**, i.e. “I'm done.”
   * If `False`, the loop should continue.

---

### ✅ Example: “terminate\_agent” Tool

You might define a tool like this:

```python
Action(
    name="terminate_agent",
    description="Indicates that the task is complete and the agent should shut down.",
    parameters={},
    terminal=True  # <--- THIS is what causes the loop to end
)
```

When the model selects this action, `should_terminate()` will return `True`, and the loop exits.

---

### 🧠 Why This Design Is Powerful

* ✅ It gives the **LLM control** over when it's done — like finishing a conversation.
* ✅ It keeps the agent loop generic — you don’t hardcode “when” to stop.
* ✅ You can have **multiple terminal tools**, like `terminate`, `cancel`, or `complete_task`.















## 🔁 Information Flow in One Loop Iteration

1. **Memory**: Supplies past conversations and results.
2. **Goals**: Define what the agent is trying to achieve.
3. **ActionRegistry**: Tells what the agent *can* do.
4. **AgentLanguage**: Converts everything into a structured prompt.
5. **LLM**: Chooses an action and returns tool + args.
6. **Environment**: Executes the selected action.
7. **Memory**: Gets updated with the result and decision.
8. **Loop**: Repeats or ends.

---

## 🧪 Creating Specialized Agents (Examples)

### 📚 Research Agent

```python
research_agent = Agent(
    goals=[Goal("Find and summarize information on topic X")],
    agent_language=ResearchLanguage(),
    action_registry=ActionRegistry([SearchAction(), SummarizeAction(), ...]),
    generate_response=openai_call,
    environment=WebEnvironment()
)
```

### 🧑‍💻 Coding Agent

```python
coding_agent = Agent(
    goals=[Goal("Write and debug Python code for task Y")],
    agent_language=CodingLanguage(),
    action_registry=ActionRegistry([WriteCodeAction(), TestCodeAction(), ...]),
    generate_response=anthropic_call,
    environment=DevEnvironment()
)
```

Each agent uses the **same loop** but behaves entirely differently due to their unique GAME components.



# What is “Environment?" I

Here’s a gentle way to think about an **Environment** in an AI agent.

---

## the simple story

* The **LLM** is the **brain**. It plans what to do: “read this file,” “search for a word,” “summarize.”
* The **Environment** is the **hands and the room** around the brain. It actually **does** the thing (opens files, runs tools) and makes sure it’s **safe**.

Think of a classroom robot:

* The robot’s brain says: “Open the math book to page 12.”
* The robot’s hands (the Environment) check: “Is this really the math book? Is it allowed to open? Okay, open it.”
  If something’s wrong, the hands say: “Nope! Not allowed,” and report back.

---

## why we need an Environment

1. **Safety** – like rules on a playground

   * “You can only play in this fenced area.”
   * For agents: “You can only read files in this folder,” “No deleting stuff,” “Only visit safe websites.”

2. **Fairness & limits** – like a timer on a game

   * “You have 10 minutes.”
   * For agents: “Stop if a tool takes too long,” “Only make a few web calls.”

3. **Organization** – like a hall monitor’s checklist

   * Keep track of what was done, how long it took, and any errors.

4. **Consistency** – like using the same whistle for every game

   * All tools return results in the **same shape** (e.g., `{ok: true, data: ...}`), so the brain can understand what happened next.

---

## what does an Environment actually do?

When the brain says:

> “Use the **read\_file** tool with `{"file_name": "notes.txt"}`”

The Environment:

1. Checks the request is safe (is “notes.txt” inside our allowed folder?).
2. Runs the tool (opens the file).
3. Wraps the result nicely (success or error).
4. Gives that back to the brain so it can plan the next step.

---

## a tiny example (very simple)

```python
class Environment:
    def __init__(self, base_dir):
        self.base_dir = base_dir  # the "fenced playground"

    def _safe_path(self, file_name):
        import os
        full = os.path.abspath(os.path.join(self.base_dir, file_name))
        if not full.startswith(os.path.abspath(self.base_dir) + os.sep):
            raise PermissionError("Not allowed outside the folder!")
        return full

    def execute_action(self, action, args):
        try:
            # simple safety rule for file tools
            if action.name in {"read_file", "search_in_file"} and "file_name" in args:
                args["file_name"] = self._safe_path(args["file_name"])

            data = action.execute(**args)  # run the tool
            return {"ok": True, "action": action.name, "data": data}
        except Exception as e:
            return {"ok": False, "action": action.name, "error": str(e)}
```

* **You can start even simpler** (just call `action.execute(**args)`), then add safety checks later.

---

## different “rooms” (environments) you might build

* **FileEnvironment**: read/search files in one safe folder.
* **WebEnvironment**: only allow certain websites; limit requests.
* **DataEnvironment**: talk to a database safely with timeouts.
* **MixedEnvironment**: combine several, each with its own rules.

---

## how it fits in your agent loop

1. Brain (LLM): “I want to run `read_file` with `{file_name: "notes.txt"}`.”
2. Agent finds that tool in your **ActionRegistry**.
3. **Environment** runs it safely and returns `{ok: true, data: ...}` or `{ok: false, error: ...}`.
4. Brain uses that result to decide the next step… or finish.

---

## key takeaways (sticky notes)

* **Brain plans; Environment does.**
* Environment is where you put **rules, safety, and logs**.
* Start with a **pass-through** version, then add seatbelts: safe paths, timeouts, and consistent results.
* Keeping this boundary clear makes your agent easier to **trust**, **debug**, and **grow**.


“Environment” is one of the trickiest ideas until it clicks. Think of it as **the boundary between the agent’s plan and the real world**.

# What is “Environment?" II

* A **runtime sandbox** that **executes actions** (tools) on behalf of the agent.
* It owns the **resources** (file system, network, DBs, APIs), **policies** (what’s allowed), and **guardrails** (validation, timeouts, rate limits).
* The LLM plans; **Environment does**—safely and predictably.

# Why not just call `action.execute(**args)` directly?

Because you want a single, central place to handle:

* **Safety**: path whitelists, URL allowlists, read-only vs write, argument sanitization.
* **Reliability**: retries, timeouts, idempotency keys, circuit breakers.
* **Observability**: logging, metrics, traces.
* **Context**: shared handles (DB connections, base folders, API clients).
* **Policy**: auth/ACLs, quotas, cost budgets.
* **Normalization**: consistent result and error shapes for the agent’s memory.

# Minimal interface (fits your loop)

Your agent calls:

```python
result = environment.execute_action(action, invocation["args"])
```

So implement `Environment` as the **execution gateway**:

```python
import json, time, os
from typing import Any, Dict

class Environment:
    def __init__(self,
                 base_dir: str | None = None,
                 allow_net: bool = False,
                 allowed_exts: set[str] = {".txt", ".md"},
                 timeout_s: float = 10.0):
        self.base_dir = os.path.abspath(base_dir) if base_dir else None
        self.allow_net = allow_net
        self.allowed_exts = allowed_exts
        self.timeout_s = timeout_s

    # ---- guardrails -------------------------------------------------
    def _assert_safe_path(self, filename: str):
        if not self.base_dir:
            return
        full = os.path.abspath(os.path.join(self.base_dir, filename))
        if not full.startswith(self.base_dir + os.sep):
            raise PermissionError("Path traversal blocked.")
        _, ext = os.path.splitext(full)
        if self.allowed_exts and ext not in self.allowed_exts:
            raise PermissionError(f"Extension {ext} not allowed.")
        return full

    # ---- execution --------------------------------------------------
    def execute_action(self, action, args: Dict[str, Any]) -> Dict[str, Any]:
        """Run the tool with safety, timing, and normalized output."""
        start = time.time()
        try:
            # Basic semantic validation examples
            if action.name in {"read_file", "search_in_file"} and "file_name" in args:
                args["file_name"] = self._assert_safe_path(args["file_name"])

            # TODO: attach timeouts/retries around the call if needed
            out = action.execute(**args)

            elapsed = time.time() - start
            return {"ok": True, "action": action.name, "args": args, "data": out, "ms": int(elapsed * 1000)}
        except Exception as e:
            elapsed = time.time() - start
            return {"ok": False, "action": action.name, "args": args, "error": str(e), "ms": int(elapsed * 1000)}
```

### Notes on this pattern

* **Guardrails before execution** (path checks, allowed extensions).
* **Normalized result shape** (`ok`, `data` or `error`, `ms`). That makes your `update_memory` simple and structured.
* You can add: **retry on transient errors**, **timeout wrappers**, **rate limits**, **network allowlist checks**, **audit logging**, etc.

# Concrete examples of “Environment”

* **FileSystemEnvironment** (like above): read-only base directory, only `.txt/.md`, deny traversal.
* **WebEnvironment**: HTTP client with domain allowlist, per-host rate limits, JSON-only responses.
* **DataEnvironment**: DB connections, transaction boundaries, query timeouts.
* **Sandboxed Python Environment**: executes small computations with resource caps (CPU, mem).
* **Orchestrated Environment**: coordinates multiple sub-environments (files + web + vector DB).

# How it works with Actions and the Registry

* **Action** = capability definition (name, description, JSON Schema, `function`).
* **Registry** = catalog to find actions by name.
* **Environment** = *executes* them with safety & policy.

Flow:

```
LLM → {tool: "read_file", args: {"file_name":"notes.txt"}}
Agent → registry.get_action("read_file")
Agent → environment.execute_action(action, args)
Environment → validates + runs action.function(**args) → returns {ok:..., data/error}
Agent → update memory → maybe next step or terminate
```

# What to focus on (key learning goals)

1. **Separation of concerns**
   Planning (LLM) vs Doing (Environment). The agent stays simple; the environment is where real-world complexity lives.

2. **Guardrails & policy**
   Treat LLM output as *untrusted*. Enforce allowlists, resource limits, and semantics (e.g., deny write ops in a read-only lecture).

3. **Deterministic outputs**
   Return structured results from tools; the environment wraps them in a consistent envelope (`ok/data/error`).

4. **Observability**
   Time each call, log args (after redaction), track failures. This is how you debug agents.

5. **Testability**
   You can swap in a **FakeEnvironment** for unit tests. The same agent loop runs, but tools return canned results—no real side effects.

# Common pitfalls

* **Letting the LLM choose any path/URL** without checks.
* **No timeouts** → stuck or expensive calls.
* **Unbounded memory** growth (environment can help by summarizing bulky results).
* **Inconsistent tool return shapes** → hard for the LLM to reason about next steps.

---

**TL;DR:**
The **Environment** is your agent’s execution sandbox. It centralizes **safety, policy, and reliability** for all tool calls. Keep it strict, observable, and consistent—and your agent loop will be far easier to reason about, debug, and scale.


Earlier snippets you shared didn’t need a separate `Environment` because they executed tools **directly**. Think of `Environment` as an optional abstraction:

* In small demos: you can just call the Python function (tool) yourself. No `Environment` needed.
* In a more complete agent (like your `Agent.run` loop): `Environment` is the **gateway** that runs tools and enforces safety/policy/logging. Same effect, cleaner and safer place to put guardrails.

### Why Environment wasn’t defined earlier

Those early lectures focused on parsing and tool selection, not execution policy. They likely omitted `Environment` for brevity and had the loop do something like:

```python
result = TOOLS[tool_name](**args)
```

All “environment” concerns (paths, timeouts, error shaping) were implicitly inside tool functions or not handled at all.

### Can the agent run without `Environment`?

Yes. Replace the call:

```python
result = self.environment.execute_action(action, invocation["args"])
```

with:

```python
result = action.execute(**invocation["args"])
```

and it will still work. You just lose a central place for:

* Safety (path/url allowlists, read-only rules)
* Reliability (timeouts, retries)
* Observability (uniform result shape, timing, logging)
* Policy (quotas, permissions, budgets)

### Minimal “pass-through” Environment (drop-in)

If your current Agent requires an `environment`, use a no-op version and evolve later:

```python
class Environment:
    def execute_action(self, action, args: dict) -> dict:
        try:
            data = action.execute(**args)
            return {"ok": True, "action": action.name, "args": args, "data": data}
        except Exception as e:
            return {"ok": False, "action": action.name, "args": args, "error": str(e)}
```

Wire it up:

```python
env = Environment()
agent = Agent(goals, agent_language, action_registry, generate_response, env)
```

### A slightly safer file-focused Environment (example)

```python
import os, time

class FileEnvironment(Environment):
    def __init__(self, base_dir: str, allowed_exts={".txt", ".md"}, timeout_s=10):
        self.base_dir = os.path.abspath(base_dir)
        self.allowed_exts = allowed_exts
        self.timeout_s = timeout_s

    def _safe_path(self, filename: str) -> str:
        full = os.path.abspath(os.path.join(self.base_dir, filename))
        if not full.startswith(self.base_dir + os.sep):
            raise PermissionError("Path traversal blocked.")
        _, ext = os.path.splitext(full)
        if self.allowed_exts and ext.lower() not in self.allowed_exts:
            raise PermissionError(f"Extension {ext} not allowed.")
        return full

    def execute_action(self, action, args: dict) -> dict:
        start = time.time()
        # Example semantic guardrails
        if action.name in {"read_file", "search_in_file"} and "file_name" in args:
            args["file_name"] = self._safe_path(args["file_name"])

        try:
            data = action.execute(**args)
            return {"ok": True, "action": action.name, "args": args, "data": data, "ms": int((time.time()-start)*1000)}
        except Exception as e:
            return {"ok": False, "action": action.name, "args": args, "error": str(e), "ms": int((time.time()-start)*1000)}
```

### Takeaways for your notes

* `Environment` isn’t an OpenAI thing; it’s your **application boundary** for executing tools.
* You can **omit it** in simple prototypes; your agent will still run.
* As soon as you care about **safety, reliability, and clean logs**, introduce `Environment` and route **all** tool calls through it.
* Keep tool functions simple; put policy/guardrails **in the environment**, not scattered across tools.
